In [2]:
from hypex.dataset.dataset import Dataset, ExperimentData
from hypex.dataset.roles import (
    InfoRole,
    FeatureRole,
    TreatmentRole,
    TargetRole,
)
from hypex.experiments.aa_test import AA_TEST
from hypex.reporters.aa import AADictReporter

In [3]:
data = Dataset(
    roles={
        "user_id": InfoRole(),
        "signup_month": FeatureRole(),
        "treat": TreatmentRole(),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole(),
        "age": FeatureRole(),
        "gender": FeatureRole(),
        "industry": FeatureRole(),
    }, data="data.csv",
)
data

      user_id  signup_month  treat  pre_spends  post_spends   age gender  \
0           0             0      0       488.0   414.444444   NaN      M   
1           1             8      1       512.5   462.222222  26.0    NaN   
2           2             7      1       483.0   479.444444  25.0      M   
3           3             0      0       501.5   424.333333  39.0      M   
4           4             1      1       543.0   514.555556  18.0      F   
...       ...           ...    ...         ...          ...   ...    ...   
9995     9995            10      1       538.5   450.444444  42.0      M   
9996     9996             0      0       500.5   430.888889  26.0      F   
9997     9997             3      1       473.0   534.111111  22.0      F   
9998     9998             2      1       495.0   523.222222  67.0      F   
9999     9999             7      1       508.0   475.888889  38.0      F   

        industry  
0     E-commerce  
1     E-commerce  
2      Logistics  
3     E-com

In [4]:
data.roles

{'user_id': Info,
 'signup_month': Feature,
 'treat': Treatment,
 'pre_spends': Target,
 'post_spends': Target,
 'age': Feature,
 'gender': Feature,
 'industry': Feature}

In [5]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [6]:
result.analysis_tables['OneAASplitAnalyzer╰╰']

   TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
0       0.173506         0.0        0.215686          0.0         0.201626

In [7]:
AADictReporter().report(result)

{'random_state': None,
 'B pre_spends control mean': 487.3426,
 'B pre_spends difference': -0.4977000000000089,
 'B pre_spends difference %': -0.10212528106511298,
 'B pre_spends test mean': 486.8449,
 'B post_spends control mean': 452.7185777777778,
 'B post_spends difference': -1.108044444444488,
 'B post_spends difference %': -0.24475347353392074,
 'B post_spends test mean': 451.6105333333333}

In [8]:
result.analysis_tables['GroupDifference╰╰post_spends[A]'].to_dict()

{'backend': 'pandas',
 'roles': {'role_names': ['B'], 'columns': [Statistic]},
 'data': {'data': {'B': [452.7185777777778,
    -1.108044444444488,
    -0.24475347353392074,
    451.6105333333333]},
  'index': ['post_spends control mean',
   'post_spends difference',
   'post_spends difference %',
   'post_spends test mean']}}

In [9]:
from hypex.experiments.ab_test import AB_TEST

test = AB_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [10]:
result.analysis_tables['ABAnalyzer╰╰']

   TTest p-value  TTest pass  MannWhitney p-value  MannWhitney pass  \
0   1.157524e-30         1.0         3.812131e-12               1.0   

   GroupATE pre_spends  GroupATE post_spends  
0            487.09375            452.164556  

In [11]:
from hypex.experiments.homogeneity_test import HOMOGENEITY_TEST

test = HOMOGENEITY_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [12]:
result.analysis_tables

{'GroupDifference╰╰pre_spends[[]':                                   1
 pre_spends control mean  484.911973
 pre_spends difference      4.308406
 pre_spends difference %    0.888492
 pre_spends test mean     489.220379,
 'TTest╰╰pre_spends[[]':    group  statistic       p-value  pass
 0      1 -11.489293  2.315047e-30  True,
 'KSTest╰╰pre_spends[[]':    group  statistic       p-value  pass
 0      1   0.077573  1.559150e-13  True,
 'GroupDifference╰╰post_spends[[]':                                    1
 post_spends control mean  420.046619
 post_spends difference     63.424045
 post_spends difference %   15.099287
 post_spends test mean     483.470664,
 'TTest╰╰post_spends[[]':    group   statistic  p-value  pass
 0      1 -135.560001      0.0  True,
 'KSTest╰╰post_spends[[]':    group  statistic  p-value  pass
 0      1     0.8959      0.0  True,
 'OneAASplitAnalyzer╰╰':    TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
 0   1.157524e-30         1.0    7.79575

In [13]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.comparators.hypothesis_testing import TTest, KSTest
from hypex.comparators.comparators import GroupSizes, GroupDifference
from hypex.utils.enums import SpaceEnum
from hypex.splitters.aa import AASplitter
from hypex.analyzers.aa import OneAASplitAnalyzer
from hypex.experiments.base import CycledExperiment, Experiment, OnRoleExperiment 

aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[ 
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional), 
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=2, inner_executor=aa, analyzer=OneAASplitAnalyzer()) 
ed = ExperimentData(data)
res = experiment.execute(ed)

In [14]:
res.analysis_tables

{'GroupSizes╰╰pre_spends[A]0':                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 'GroupDifference╰╰pre_spends[A]0':                                   B
 pre_spends control mean  487.342600
 pre_spends difference     -0.497700
 pre_spends difference %   -0.102125
 pre_spends test mean     486.844900,
 'TTest╰╰pre_spends[A]0':   group  statistic   p-value   pass
 0     B   1.318771  0.187276  False,
 'KSTest╰╰pre_spends[A]0':   group  statistic   p-value   pass
 0     B      0.021  0.220219  False,
 'GroupSizes╰╰post_spends[A]0':                      B
 control size    5000.0
 control size %    50.0
 test size       5000.0
 test size %       50.0,
 'GroupDifference╰╰post_spends[A]0':                                    B
 post_spends control mean  452.718578
 post_spends difference     -1.108044
 post_spends difference %   -0.244753
 post_spends test mean     451.610533,
 'TTest╰╰post_spends[A]0':   group  statistic   

In [15]:
res_analyzer = OneAASplitAnalyzer().execute(res)

In [16]:
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']

   TTest p-value  TTest pass  KSTest p-value  KSTest pass  mean test score
0       0.173506         0.0        0.215686          0.0         0.201626

In [17]:
# analyser version

from hypex.dataset.roles import StatisticRole

new_res = {}
fields = ['pre_spends', 'post_spends'] # получаем столбцы по TargetRole 
tests = ['KSTest', 'TTest']
for j in fields:
    for i in list(res.analysis_tables.keys()):  
        if j in i and i.split('╰╰')[0] in tests: 
            test = i.split('╰╰')[0]
            if j not in new_res: 
                new_res[j] = {test: []}
            elif test not in new_res[j]: 
                new_res[j].update({test: []}) 
            new_res[j][test].append(list(res.analysis_tables[i].data['pass'])[0])
    new_res[j] = Dataset.from_dict(new_res[j], roles={}).mean()
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['TTest'] <= 1.2 * 0.05), {'TTest passed': StatisticRole()}, axis=1
            ))
    new_res[j].add_column(new_res[j].apply(
                lambda x: int(0.8 * 0.05 <= x['KSTest'] <= 1.2 * 0.05), {'KSTest passed': StatisticRole()}, axis=1
            ))
new_res

{'pre_spends':       KSTest  TTest  TTest passed  KSTest passed
 mean     0.0    0.0             0              0,
 'post_spends':       KSTest  TTest  TTest passed  KSTest passed
 mean     0.0    0.0             0              0}